交叉特征

In [2]:
# 使用tf.keras搭建回归模型，数据集使用加利福尼亚的房价预测
#导入必要的库即版本

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
#import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.16.4
pandas 0.23.4
sklearn 0.19.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [3]:
# 数据文件路径
train_file = './data/titanic/train.csv'
eval_file = './data/titanic/eval.csv'

# 读取数据
train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

In [4]:
# 提取特征和标签
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

In [5]:
# 将特征分为离散特征和连续特征
categorical_columns = ['sex', 'n_siblings_spouses','parch', 'class', 'deck', 
                      'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []
for categorical_column in categorical_columns:
    # 获取单独特征所有可能的值
    vocab = train_df[categorical_column].unique()
    print(categorical_column,vocab)
    # 进行one-hot编码,tf.feature_column.indicator_column
    feature_columns.append(tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            categorical_column,vocab)))
    
for numeric_column in numeric_columns:
    # 连续特征，直接加载
    feature_columns.append(
        tf.feature_column.numeric_column(
            numeric_column,dtype=tf.float32))
    
# cross feature
# 交叉特征
feature_columns.append(tf.feature_column.indicator_column(
    tf.feature_column.crossed_column(['age','sex'],
                                hash_bucket_size=100)))

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


CrossedColumn(keys=('age', 'sex'), hash_bucket_size=100, hash_key=None)

In [6]:
# 构建dataset
def make_dataset(data_df, label_df, epochs=10, shuffle=True,
                batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
        
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [8]:
# 文件夹保存中间输出的模型
output_dir = 'baseline_model'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
baseline_estimator = tf.estimator.BaselineClassifier(
    model_dir=output_dir,
    n_classes=2
)
baseline_estimator.train(input_fn=lambda : make_dataset(
    train_df,y_train,epochs=100))

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at <ipython-input-8-7ae19b05e39f>:10) ]]

Original stack trace for 'IteratorGetNext':
  File "D:\SoftWare\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\SoftWare\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\SoftWare\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\SoftWare\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\SoftWare\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "D:\SoftWare\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_

In [11]:
baseline_estimator.evaluate(input_fn=lambda:make_dataset(
    eval_df,y_eval,epochs=1,shuffle=False))

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[node IteratorGetNext (defined at <ipython-input-11-d9dfae3365cf>:1) ]]

Original stack trace for 'IteratorGetNext':
  File "D:\SoftWare\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\SoftWare\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\SoftWare\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\SoftWare\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\SoftWare\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "D:\SoftWare\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_

In [7]:
linear_output_dir = 'linear_model_new_feature'
if not os.path.exists(linear_output_dir):
    os.mkdir(linear_output_dir)
    
linear_estimator = tf.estimator.LinearClassifier(
    model_dir=linear_output_dir,
    n_classes=2,
    feature_columns=feature_columns)

linear_estimator.train(input_fn=lambda:make_dataset(train_df,y_train,epochs=100))

W0817 15:58:26.700389  2408 deprecation.py:323] From D:\SoftWare\Anaconda3\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0817 15:58:27.212020  2408 deprecation.py:323] From D:\SoftWare\Anaconda3\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0817 15:58:27.229977  2408 deprecation.py:323] From D:\SoftWare\Anaconda3\lib\site-packages\tensorflow\python\feature_column\feature_column_v2.py:4215: IndicatorColumn._variable_shape (from tensorflow.pytho

In [8]:
#tensorboard命令：tensorboard --logdir=./

linear_estimator.evaluate(input_fn=lambda:make_dataset(
    eval_df,y_eval,epochs=1,shuffle=False))

W0817 15:58:33.273340  2408 deprecation.py:323] From D:\SoftWare\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


{'accuracy': 0.78409094,
 'accuracy_baseline': 0.625,
 'auc': 0.83844507,
 'auc_precision_recall': 0.7847754,
 'average_loss': 0.46948257,
 'label/mean': 0.375,
 'loss': 0.45356736,
 'precision': 0.7058824,
 'prediction/mean': 0.39596,
 'recall': 0.72727275,
 'global_step': 1960}

In [9]:
dnn_output_dir = './dnn_model_new_feature'
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
dnn_estimator = tf.estimator.DNNClassifier(
    model_dir=dnn_output_dir,
    n_classes=2,
    feature_columns=feature_columns,
    hidden_units=[128,128],
    activation_fn=tf.nn.relu,
    optimizer='Adam')
dnn_estimator.train(input_fn=lambda:make_dataset(
    train_df,y_train,epochs=100))

In [10]:
dnn_estimator.evaluate(input_fn=lambda:make_dataset(
    eval_df,y_eval,epochs=1,shuffle=False))

{'accuracy': 0.8030303,
 'accuracy_baseline': 0.625,
 'auc': 0.8495561,
 'auc_precision_recall': 0.8182549,
 'average_loss': 0.45782208,
 'label/mean': 0.375,
 'loss': 0.44224137,
 'precision': 0.7196262,
 'prediction/mean': 0.40807644,
 'recall': 0.7777778,
 'global_step': 1960}